In [11]:
import numpy as np
from collections import Counter
import porter as p

# Exercice 1

In [60]:
docs=["the new home has been saled on top forecasts",
     "the home home sales rise in july",
     "there is an increase in home sales in july",
     "july encounter a new home sales rise"]

stopWords=["the","a","an","on","behind","under","there","in"]

index = {}
for i in range(len(docs)):
    index[i] = dict(Counter(map(p.stem, [word for word in (str.lower(docs[i])).split() if word not in stopWords])))           
            
indexInverse = {}
for numDoc, dico in index.items():
    for word, tf in dico.items():
        if(word not in indexInverse):
            indexInverse[word]= {}
        indexInverse[word][numDoc] = tf

Ordonnancement :

pour VECTEUR
- Utiliser index inverse pour récup pool de doc
- ne pas mettre en vecteur els doc et la requete mais seulement comaprer les bon terme

In [13]:
#Modele boolean
req_str="home sales top top"
req = list(np.unique(list(map(p.stem, req_str.split()))))
res=set(index)
for stem in req:
    res=res.intersection(indexInverse[stem])
res

{0}

In [14]:
dict(Counter(np.unique(list(map(p.stem, req_str.split())))))


{'home': 1, 'sale': 1, 'top': 1}

In [95]:
class Weighter():
    
    def __init__(self, index, indexInverse):
        self.index = index
        self.indexInverse = indexInverse
        self.idf = {}
        self.norm = {}
        
    def getIdf(self, stem):
        if(stem not in self.idf):
            self.idf[stem] = np.log((1+len(self.index)) / (1+len(indexInverse[stem])))
        return self.idf[stem]
    
    def getWeightsForDoc(self, idDoc):
        pass
    
    def getWeightsForStem(self, stem):
        pass
    
    def getWeightsForQuery(self, query):
        pass
    
    def getNormDoc(self, docId):
        #RAPPEL : norm([1,2,3])= norm([0,0,1,2,3,0]) car on va mettre au carré chaque element et le sommer.
        #On vectorise le document 
        if(docId not in self.norm):
            docWeights = self.getWeightsForDoc(docId)#On récupère le poids de chaque terme dans doc
            self.norm[docId] = np.linalg.norm(list(docWeights.values()))#On récupère sous forme de list et on calcul la norm 
        return self.norm[docId]
    
    def getNormQuery(self, query):
        #CF explication fonction getNormDoc
        reqWeights = self.getWeightsForQuery(query)
        return np.linalg.norm(list(reqWeights.values()))
    
    
class Weighter1(Weighter):
    
    def getWeightsForDoc(self, idDoc):
        return index[idDoc]
    
    def getWeightsForStem(self, stem):
        return indexInverse[stem]
    
    def getWeightsForQuery(self, query):
        return dict(Counter(np.unique(list(map(p.stem, query.split())))))
    
class Weighter2(Weighter):
    
    def getWeightsForDoc(self, idDoc):
        return index[idDoc]
    
    def getWeightsForStem(self, stem):
        return indexInverse[stem]
    
    def getWeightsForQuery(self, query):
        return dict(Counter(list(map(p.stem, query.split()))))
    
class Weighter3(Weighter):
    
    def getWeightsForDoc(self, idDoc):
        return index[idDoc]
    
    def getWeightsForStem(self, stem):
        return indexInverse[stem]
    
    def getWeightsForQuery(self, query):
        req=np.unique(list(map(p.stem, query.split())))
        res={}
        for stem in req:
            res[stem] = self.getIdf(stem)
        return res
    
    
class Weighter4(Weighter):
    
    def getWeightsForDoc(self, idDoc):
        res={}
        for stem in index[idDoc]:
            res[stem] = 1+np.log(index[idDoc][stem])
        return res
    
    def getWeightsForStem(self, stem):
        res={}
        for doc in indexInverse[stem]:
            res[doc] = 1+np.log(indexInverse[stem][doc])
        return res
    
    def getWeightsForQuery(self, query):
        req=np.unique(list(map(p.stem, query.split())))
        res={}
        for stem in req:
            res[stem] = self.getIdf(stem)
        return res
    

class Weighter5(Weighter):
    
    def getWeightsForDoc(self, idDoc):
        res = {}
        for stem in index[idDoc]:
            idf = self.getIdf(stem)
            res[stem] = (1+np.log(index[idDoc][stem])) * idf
        return res
    
    def getWeightsForStem(self, stem):
        res={}
        idf = self.getIdf(stem)
        for doc in indexInverse[stem]:
            res[doc] = (1+np.log(indexInverse[stem][doc])) * idf
        return res
    
    def getWeightsForQuery(self, query):
        tfs=dict(Counter(list(map(p.stem, query.split()))))
        res={}
        for stem in tfs:
            idf = self.getIdf(stem)
            res[stem] = (1+np.log(tfs[stem]))*idf
        return res

## Remarque :

Le poids d'un terme n'appartenant pas à la requete sera toujours nul. Ainsi, le produit scalaire entre le vecteur de la requete et un vecteur autre ne prendra pas en compte les termes ne se trouvant pas dans la requete (multiplication par 0).

Ainsi, on ne retournera pas les documents ayant un score nul (rapidité d'execution). La norme de chaque vecteur sera calculé la première fois que cela est nécessaire et garder en mémoire pour la suite.

Le cosinus (= produit_scalaire(A,B) / norm(A)\*norm(B)) ne peut etre négatif que si le produit scalaire l'est. Or das notre cas le produit scalaire sera toujours positif car les poids des termes sont toujours positifs.

In [16]:
"""
TODO :
- Faire une seule fonction getScores car il y a des redondances donc pas terribles !!! 
(on a besoin pour le cosinus d'infos comme poids de la requete qu'on a deja calculé dans fonction produit scalaire.
en pus on réitère sur le res du produit scalaire alors qu'on pourrait faire calcul dans le prod scalaire)
"""
class IRModel():
    
    def getScores(query):
        pass
    
    def getRanking(query):
        return sorted(list(zip(query.keys(),query.values())), key=lambda e : e[1])

class Vectoriel(IRModel):
    
    def __init__(self, weighter, normalized):
        self.weighter = weighter
        self.normalized = normalized
        
    def getScores(self, query):
        if(self.normalized == True):#Cosinus
            return getScoresNormalized(query)
        else:#produit scalaire
            return getScoresNotNormalized(query)
        
    def getScoresNormalized(self, query):
        """
        Permet de récupérer le score en utilisant le cosinus entre les représentations vectorielles des documents
        """
        #cos(A,B)= A.B / (||A||.||B||) où v.w est le produit scalaire de v et w et ||v|| est la norme de v
        prodScalaires = self.getScoresNotNormalized(query)#Le produit scalaire est le résultat de l'autre fonction
        res={}
        for docId,prod in prodScalaires.items():#Pour chaque produit scalaire, on divise par le produit des norm
            res[docId] = prod/(self.weighter.getNormDoc(docId)*self.weighter.getNormQuery(query))
        return res
    
    def getScoresNotNormalized(self, query):
        """
        Permet de récupérer le score en utilisant le produit scalaire entre les représentations vectorielles des documents
        """
        reqWeights = self.weighter.getWeightsForQuery(query)
        res={}
        for stem,weightStem in reqWeights.items():
            docWeights = self.weighter.getWeightsForStem(stem)
            for docId,weightDoc in docWeights.items():
                if(docId not in res):
                    res[docId] = weightStem*weightDoc
                else:
                    res[docId] += weightStem*weightDoc
        #Mono ligne avec dictionnaire de comprehension ?         
        return res    

In [17]:
query={1:30,2:20}
sorted(list(zip(query.keys(),query.values())), key=lambda e : e[1], reverse=True)

[(1, 30), (2, 20)]

In [18]:
d={1:10,2:20,3:30}
for (k,v) in d.items():
    print("k:",k," | v:",v)

k: 1  | v: 10
k: 2  | v: 20
k: 3  | v: 30


In [19]:
np.linalg.norm(list(d.values()))

37.416573867739416

In [20]:
[i if i%2==0 else i+1 for i in range(0,10) ]

[0, 2, 2, 4, 4, 6, 6, 8, 8, 10]